<a href="https://colab.research.google.com/github/YanhuaZhang516/memory-representation-pomdp/blob/main/embedding_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed,Flatten, Conv1D,MaxPooling1D, Bidirectional
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd
from ast import literal_eval
from keras.layers import Input, Reshape

In [ ]:
import matplotlib.pyplot as plt
import keras
keras.callbacks.Callback()
from keras import regularizers

### load the data

In [ ]:
df2 = pd.read_csv('trajectory_10_1000_grids_pos_new.csv')
df2 = df2.apply(pd.to_numeric, errors='ignore')

In [ ]:
def generate_batch_data(time_steps, num_example,df):
  batch_inputs=[]
  batch_labels=[]

  for i in range(num_example):
    # batch_size
    inputs=[]
    labels=[]
    
    for j in range(time_steps):
      # time_steps
      action= np.float32(df.loc[df['trajectory']==(i+1)]['inputs'][100*i+j][1])
      obs = literal_eval(df.loc[df['trajectory']==(i+1)]['obs_matrix'][100*i+j])
      
      state = np.float32(df.loc[df['trajectory']==(i+1)]['state'][100*i+j])
     
      label = to_categorical(state, 100)
   

      input=np.hstack((action, obs))
    
      inputs.append(input)
      labels.append(label)
 
    inputs=np.array(inputs).reshape((time_steps,10))
    labels = np.array(labels).reshape((time_steps,100))
    
    batch_inputs.append(inputs)
    
    batch_labels.append(labels)


  return batch_inputs, batch_labels

In [ ]:
batch_inputs, batch_labels =  generate_batch_data(time_steps=100, num_example=100,df=df2)

In [ ]:
batch_inputs=np.array(batch_inputs).reshape(100,100,10)
batch_labels = np.array(batch_labels).reshape(100,100,100)

In [ ]:
x_train = batch_inputs[:75]
y_train_label = batch_labels[:75]
x_val=batch_inputs[75:]
y_val_label=batch_labels[75:]

In [ ]:
print(len(x_train))
print(x_train[0].shape)
print(len(y_train_label))
print(y_train_label[0].shape)
print(len(x_val))

75
(100, 10)
75
(100, 100)
25


In [ ]:
from keras import Model

In [ ]:

input=Input(shape=(100,10,))
# print(input.shape)
output1=Embedding(35,60,input_length=100)(input)
#model.add(Embedding(35,60,input_length=100))
# print("output after embedding:",output1.shape)
output1 = Reshape((100,600))(output1)
# print(output1.shape)
output2=(LSTM(units=50, return_sequences=True))(output1)
# print(output2.shape)
output3 =(TimeDistributed(Dense(100)))(output2)
print(output3.shape)
output4=(Activation('softmax'))(output3)
print(output4.shape)
model= Model(inputs=input,outputs=output4)
model.compile(loss="categorical_crossentropy",metrics=['categorical_accuracy'], optimizer='adam')
print(model.summary())

(None, 100, 100)
(None, 100, 100)
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 100, 10)]         0         
_________________________________________________________________
embedding_32 (Embedding)     (None, 100, 10, 60)       2100      
_________________________________________________________________
reshape_13 (Reshape)         (None, 100, 600)          0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 100, 50)           130200    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 100, 100)          5100      
_________________________________________________________________
activation_10 (Activation)   (None, 100, 100)          0         
Total params: 137,400
Trainable params: 137,400
Non-trainable params: 0
___________________

In [ ]:
model.fit(x= batch_inputs[:99], y= batch_labels[:99],validation_split=0.3,epochs=300, batch_size=10)

Epoch 1/300
7/7 [==============================] - 4s 208ms/step - loss: 4.5947 - categorical_accuracy: 0.0325 - val_loss: 4.5274 - val_categorical_accuracy: 0.0537
Epoch 2/300
7/7 [==============================] - 1s 119ms/step - loss: 4.5032 - categorical_accuracy: 0.0597 - val_loss: 4.3681 - val_categorical_accuracy: 0.0537
Epoch 3/300
7/7 [==============================] - 1s 118ms/step - loss: 4.3366 - categorical_accuracy: 0.0558 - val_loss: 4.1997 - val_categorical_accuracy: 0.0537
Epoch 4/300
7/7 [==============================] - 1s 123ms/step - loss: 4.2058 - categorical_accuracy: 0.0493 - val_loss: 4.0468 - val_categorical_accuracy: 0.0537
Epoch 5/300
7/7 [==============================] - 1s 119ms/step - loss: 4.0517 - categorical_accuracy: 0.0547 - val_loss: 3.8939 - val_categorical_accuracy: 0.0580
Epoch 6/300
7/7 [==============================] - 1s 123ms/step - loss: 3.8841 - categorical_accuracy: 0.0637 - val_loss: 3.7524 - val_categorical_accuracy: 0.0853
Epoch 7/30

In [ ]:
test_x= batch_inputs[99].reshape((-1,100,10))
pred_y_label = model.predict(test_x)
predicted = np.argmax (pred_y_label, axis=-1 )
test_y_label = np.argmax (test_y, axis=-1 )

In [ ]:
print(test_y_label)

[12 12 22 12 13 14 15 15 25 24 25 15 16 26 25 15 16 16 16 16 17 16 16 15
 15 25 24 23 22 22 12 13 23 24 23 33 34 24 34 33 43 42 42 43 44 43 33 43
 33 32 32 32 33 32 32 42 32 42 52 62 62 72 73 72 82 82 83 82 82 72 73 74
 64 63 53 54 55 56 46 36 35 34 33 32 42 42 43 42 43 33 34 24 25 35 36 37
 47 57 47 48]


In [ ]:
print(predicted)

[[12 12 22 12 13 14 15 15 25 24 25 15 16 26 25 15 16 16 16 16 17 16 16 15
  15 25 24 23 22 22 12 13 23 24 23 33 34 24 34 33 43 42 42 43 44 43 33 43
  33 32 32 32 33 32 32 42 32 42 52 62 62 72 73 72 82 82 83 82 82 72 73 74
  64 63 53 54 55 56 46 36 35 34 33 32 42 42 43 42 43 33 34 24 25 35 36 37
  47 57 47 48]]


In [ ]:
print(predicted-test_y_label)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
model.fit(x= batch_inputs[:99], y= batch_labels[:99],validation_split=0.3,epochs=100, batch_size=10)

Epoch 1/100
7/7 [==============================] - 1s 136ms/step - loss: 0.1396 - categorical_accuracy: 0.9986 - val_loss: 0.2320 - val_categorical_accuracy: 0.9597
Epoch 2/100
7/7 [==============================] - 1s 120ms/step - loss: 0.1384 - categorical_accuracy: 0.9986 - val_loss: 0.2295 - val_categorical_accuracy: 0.9547
Epoch 3/100
7/7 [==============================] - 1s 120ms/step - loss: 0.1378 - categorical_accuracy: 0.9987 - val_loss: 0.2409 - val_categorical_accuracy: 0.9537
Epoch 4/100
7/7 [==============================] - 1s 122ms/step - loss: 0.1367 - categorical_accuracy: 0.9986 - val_loss: 0.2276 - val_categorical_accuracy: 0.9537
Epoch 5/100
7/7 [==============================] - 1s 125ms/step - loss: 0.1356 - categorical_accuracy: 0.9987 - val_loss: 0.2217 - val_categorical_accuracy: 0.9587
Epoch 6/100
7/7 [==============================] - 1s 120ms/step - loss: 0.1348 - categorical_accuracy: 0.9987 - val_loss: 0.2207 - val_categorical_accuracy: 0.9587
Epoch 7/10

In [ ]:
test_x= batch_inputs[99].reshape((-1,100,10))
pred_y_label = model.predict(test_x)
predicted = np.argmax (pred_y_label, axis=-1 )
test_y_label = np.argmax (test_y, axis=-1 )

In [ ]:
print(test_y_label)

[12 12 22 12 13 14 15 15 25 24 25 15 16 26 25 15 16 16 16 16 17 16 16 15
 15 25 24 23 22 22 12 13 23 24 23 33 34 24 34 33 43 42 42 43 44 43 33 43
 33 32 32 32 33 32 32 42 32 42 52 62 62 72 73 72 82 82 83 82 82 72 73 74
 64 63 53 54 55 56 46 36 35 34 33 32 42 42 43 42 43 33 34 24 25 35 36 37
 47 57 47 48]


In [ ]:
print(predicted)

[[12 12 22 12 13 14 15 15 25 24 25 15 16 26 25 15 16 16 16 16 17 16 16 15
  15 25 24 23 22 22 12 13 23 24 23 33 34 24 34 33 43 42 42 43 44 43 33 43
  33 32 32 32 33 32 32 42 32 42 52 62 62 72 73 72 82 82 83 82 82 72 73 74
  64 63 53 54 55 56 46 36 35 34 33 32 42 42 43 42 43 33 34 24 25 35 36 37
  47 57 47 48]]


In [ ]:
print(predicted-test_y_label)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
